# Final Project
## Introduction to Business Analytics
Spring 2017 
***

### Introduction

Lending Club is an online credit marketplace that facilitates peer to peer lending by matching up investors and borrowers. Potential borrowers sign up to Lending Club and go through a quick approval process that allows them to request a personal, business, or medical loan.  Investors then make offers to borrowers based on their credit ratings and Lending Club’s suggested interest rates. Borrowers then begin to accept offers and have 15 days to reach a cumulative amount of at least 75% of their requested loan. The risk assessment process is automated and borrowers can begin to receive loan offers within minutes. (Lendingclub.com)

### The Business Problem

Unlike most banks and traditional lenders, Lending Club has a very flexible prepayment policy. Borrowers can prepay their loan at any time and without incurring a fee. Furthermore, investor fees may be reduced depending on the timing of the prepayment. Investors pay a payment processing fee equal to 1% of their monthly payment. If the loan is prepaid within the first 12 months then the investor only pays 1% of the contractual monthly payment and no fee is paid on the prepaid portion of the loan. However, if the loan is prepaid after the first 12 months then the investor pays a fee of 1% of the entire payment. In other words, if the borrower prepays during the first 12 months, Lending Club will only receive up to 33% and 20% of the total fee on a three- and five-year loan, respectively. If a loan is prepaid after the first 12 months then Lending Club will receive the entire fee. (For simplicity, we assume that any prepayments done during the first 12 months occur at the end of the 12th month.)

Prepayments can represent a significant loss in revenue for Lending Club. In its last 10-K, Lending Club reported service fees of 11.5m, 32.8m, and 68.0m dollars for 2014, 2015, and 2016, respectively. Additionally, data analysis shows that approximately X% of loans initiated in 201X were prepaid. This translates into a loss of up to XXM dollars for Lending club in 201X.
    
As Lending Club grows, the number of loans increase as do the number of prepayments. One solution to the problem is to implement a dynamic prepayment policy that would charge the borrower a prepayment fee based on the probability that the borrower prepays the loan. Lending club could maintain the 0% fee for borrowers less likely to prepay and charge a 1% fee for borrowers more likely to prepay. That way, in addition to protecting itself from a loss, Lending Club would make up to 33% more in fees on loans that were prepaid within the first 12 months.

### Notes

The Data

•	Please provide some notes (bullets are fine) on the features we selected… I’ll add the discussion about the timing

The Target Variable

•	Please provide the details on how we derive the target feature from current features.

Notes for profit curve to be done later:
1.	Assume 100 dollar loan
2.	Assume a probability that a borrower will not take out a loan if they have to pay a prepayment fee – this will represent a potential loss
3.	Assume a fee of 0% or 1%

Short cut cheatsheet: https://gist.github.com/kidpixo/f4318f8c8143adee5b40

## Approved Applications

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt

# For file loading and memory monitoring
import os
import gc
import psutil
'''
    If you get error saying no module named psutil, run this in terminal :
    sudo su
    pip install psutil
'''

# collect garbage and check current memory
def collect_and_check_mem():
    proc = psutil.Process(os.getpid())
    gc.collect()
    mem = proc.memory_info().rss
    print ("Memory : %.2f MB" % (mem / (1000 * 1000)))
collect_and_check_mem()

Memory : 81.36 MB


In [2]:
# data loading util
def load_year_data(year, suffixs):
    loans_data = pd.DataFrame()
    for index in range(len(suffixs)):
        suffix = suffixs[index]
        path ="data/loans/%s/xa%s.csv.gz" % (year, suffix)
        if index == 0:
            loans_data = pd.read_csv(path, skiprows=1)
        else:
            frame = pd.read_csv(path, skiprows=0, names=loans_data.columns)
            loans_data = loans_data.append(frame, ignore_index=True)
            del frame
    return loans_data
# create suffix from start (e.g. 'a') to end (e.g. 'z')
def create_suffixs(start, end):
    return [chr(i) for i in range(ord(start), ord(end)+1)]

In [3]:

#df1 = pd.read_csv("data/loans/2007-2011/xaa.csv.gz",skiprows=1)
#df2 = pd.read_csv("data/loans/2007-2011/xab.csv.gz",skiprows=0, names=df1.columns)
#loans_2007_2011 = pd.concat([df1,df2])
#loans_2007_2011.columns

#Reason why memory overflow: 
#    When we do read and concat, concat actually create a copy with each data frame (data frame is immutable)
#    So after concat at least we need to delete the data chunks and do garbage collection:
#        del df1
#        gc.collect()
#    But never mind, we will use load_year_data helper which will load data for each year without keeping temporary data 

# 'a' to 'b' for 2007-2011
# suffixs = create_suffixs('a', 'b')
# loans_2007_2011 = load_year_data('2007-2011', suffixs)


In [4]:
# 'a' to 'g' for 2012-2013
# suffixs = create_suffixs('a', 'g')
# loans_2007_2011 = load_year_data('2012-2013', suffixs)

In [5]:
# 'a' to 'h' for 2014
# suffixs = create_suffixs('a', 'h')
# loans_2014 = load_year_data('2012-2013', suffixs)

In [6]:
# 'a' to 'o' for 2015
suffixs = create_suffixs('a', 'o')
loans_2015 = load_year_data('2015', suffixs)
collect_and_check_mem()
len(loans_2015.index)

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (55) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


Memory : 584.86 MB


421097

In [7]:
# Before scaling up the instance we can try small set of data
# years_info = {
#     "2016Q1": create_suffixs('a', 'e'), 
#     #"2016Q2": create_suffixs('a', 'd'), 
#     #"2016Q3": create_suffixs('a', 'd'), 
#     "2016Q4": create_suffixs('a', 'd')
# }
# loans_2016 = pd.DataFrame()
# for year in years_info:
#     frame = load_year_data(year, years_info[year])
#     loans_2016 = loans_2016.append(frame, ignore_index=True)
#     collect_and_check_mem()
# collect_and_check_mem()
# len(loans_2016.index)

In [8]:
# # Merge any years you want
# loans_data = pd.concat([loans_2015, loans_2016])
# collect_and_check_mem()

# # IMPORTANT: remove the useless temporary frames:
# del loans_2015
# del loans_2016
# collect_and_check_mem()
# len(loans_data.index)

# While the collected memory of deleted object will not be returned to OS but kept for python
# so the memory does not go down as expected, but actually they are available:
# reference: http://stackoverflow.com/questions/39100971/how-do-i-release-memory-used-by-a-pandas-dataframe

In [9]:
# Rename it to loans_data
loans_data = loans_2015

In [10]:
# since too many features, maybe we take out those we wanna keep:
features_to_keep = set([
    # numerical
    'loan_amnt', 'funded_amnt', 'annual_inc', 'installment',
    'open_acc', 'total_acc',
        # Some of the following line features are duplicates? Do we need them all?
        # Say: total_pymnt = total_rec_prncp + total_rec_int
    'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee',
    'recoveries', 'collection_recovery_fee', 
    'last_pymnt_amnt',  
    
    # features used to extract target and drop once that done
    'loan_status',
    'last_pymnt_d', # used to extract target
    'issue_d', # categorical ? Dec-11
    
    # categorical
        #date
    'last_credit_pull_d', # 21 NULL values for 2016Q1Q4 same as above, maybe ignore it
        #other
    'verification_status', 'purpose', 'addr_state',
    'grade', 'sub_grade', 'home_ownership', 'term',
    
    # special
    #'title', # 10629 NULLs for 2016 Q1Q4 value maybe forget it
    
    'int_rate', 'revol_util' # trim out percentage mark: 10.65%
    'emp_length', # extracting number: 10+ years < 1 year
    
    # not sure:
    'inq_last_6mths', 'pub_rec', 'revol_bal', 'dti', 'delinq_2yrs', 
    'pymnt_plan', 'earliest_cr_line' 'initial_list_status',
    'out_prncp', 'out_prncp_inv',
    'collections_12_mths_ex_med',
    'policy_code', 'application_type',
    'acc_now_delinq', 'chargeoff_within_12_mths', 'delinq_amnt',
    'pub_rec_bankruptcies', 'tax_liens'
])

for column in loans_data.columns:
    if column not in features_to_keep:
        loans_data = loans_data.drop(column, axis=1)
loans_data.columns

Index(['loan_amnt', 'funded_amnt', 'term', 'int_rate', 'installment', 'grade',
       'sub_grade', 'home_ownership', 'annual_inc', 'verification_status',
       'issue_d', 'loan_status', 'pymnt_plan', 'purpose', 'addr_state', 'dti',
       'delinq_2yrs', 'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal',
       'total_acc', 'out_prncp', 'out_prncp_inv', 'total_pymnt',
       'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int',
       'total_rec_late_fee', 'recoveries', 'collection_recovery_fee',
       'last_pymnt_d', 'last_pymnt_amnt', 'collections_12_mths_ex_med',
       'policy_code', 'application_type', 'acc_now_delinq',
       'chargeoff_within_12_mths', 'delinq_amnt', 'pub_rec_bankruptcies',
       'tax_liens'],
      dtype='object')

In [11]:
# Check NULL
loans_data.isnull().sum()

loan_amnt                       2
funded_amnt                     2
term                            2
int_rate                        2
installment                     2
grade                           2
sub_grade                       2
home_ownership                  2
annual_inc                      2
verification_status             2
issue_d                         2
loan_status                     2
pymnt_plan                      2
purpose                         2
addr_state                      2
dti                             2
delinq_2yrs                     2
inq_last_6mths                  2
open_acc                        2
pub_rec                         2
revol_bal                       2
total_acc                       2
out_prncp                       2
out_prncp_inv                   2
total_pymnt                     2
total_pymnt_inv                 2
total_rec_prncp                 2
total_rec_int                   2
total_rec_late_fee              2
recoveries    

In [13]:
# Drop rows with NULL values:
loans_data = loans_data.dropna()
total_num = len(loans_data.index)
#loans_data.isnull().sum()
total_num

420801

In [14]:
loans_data["term"].unique()

array([' 60 months', ' 36 months'], dtype=object)

In [15]:
loans_data["last_pymnt_d"].unique()

array(['Feb-2017', 'Jun-2016', 'Oct-2016', 'Jul-2016', 'Nov-2016',
       'Jan-2017', 'Dec-2016', 'Apr-2016', 'Aug-2016', 'Mar-2016',
       'Sep-2016', 'May-2016', 'Feb-2016', 'Jan-2016', 'Dec-2015',
       'Nov-2015', 'Oct-2015', 'Sep-2015', 'Aug-2015', 'Jul-2015',
       'Jun-2015', 'May-2015', 'Apr-2015', 'Mar-2015', 'Feb-2015',
       'Jan-2015'], dtype=object)

In [16]:
loans_data["issue_d"].unique()

array(['Dec-2015', 'Nov-2015', 'Oct-2015', 'Sep-2015', 'Aug-2015',
       'Jul-2015', 'Jun-2015', 'May-2015', 'Apr-2015', 'Mar-2015',
       'Feb-2015', 'Jan-2015'], dtype=object)

In [17]:
loans_data["loan_status"].unique()

array(['Current', 'Fully Paid', 'Default', 'Charged Off',
       'Late (16-30 days)', 'Late (31-120 days)', 'In Grace Period'], dtype=object)

In [19]:
from datetime import datetime
from random import randint

fully_paid = 0
pre_paid = 0 # within 366 days
pre_paid_relaxed = 0
Y = [] # pre_paid : 1   not pre_paid : 0
sample = 5
for index in range(total_num):
    df = loans_data.iloc[index]
    if df["loan_status"] == "Fully Paid":
        fully_paid += 1
        time1 = datetime.strptime(df["issue_d"], '%b-%Y')
        time2 = datetime.strptime(df["last_pymnt_d"], '%b-%Y')
        diff = abs(time1 - time2).days
        if diff < 366:
            pre_paid += 1
        if diff < 400:
            pre_paid_relaxed += 1
        if sample > 0 and randint(0, total_num) < 10:
            collect_and_check_mem()
            print(time1)
            print(time2)
            print(diff)
            sample -= 1
            
        Y.append(1 if diff < 400 else 0)
print("Prepaid: %d out of Fully Paid: %d" % (pre_paid, fully_paid))

Memory : 666.29 MB
2015-11-01 00:00:00
2016-11-01 00:00:00
366
Memory : 666.29 MB
2015-10-01 00:00:00
2016-01-01 00:00:00
92
Memory : 666.29 MB
2015-04-01 00:00:00
2015-06-01 00:00:00
61
Prepaid: 50629 out of Fully Paid: 94968


In [20]:
pre_paid_relaxed

61854

## Rejected applications (deprecated)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as plt

reject_2007_2012 = pd.read_csv("data/rejected/2007-2012/RejectStatsA.csv.gz",skiprows=1)

#df1 = pd.read_csv("data/rejected/2013-2014/xaa.csv.gz",skiprows=1)
#df2 = pd.read_csv("data/rejected/2013-2014/xab.csv.gz",skiprows=0,names=df1.columns)
#reject_2013_2014 = pd.concat([df1,df2])

#df2 = pd.read_csv("data/rejected/2015/xaa.csv.gz",skiprows=1)
#df2 = pd.read_csv("data/rejected/2015/xab.csv.gz",skiprows=0, names=df1.columns)
#reject_2015 = pd.concat([df1,df2])

#reject_2016Q1 = pd.read_csv("data/rejected/2016Q1/RejectStats_2016Q1.csv.gz",skiprows=1)

#reject_2016Q2 = pd.read_csv("data/rejected/2016Q2/RejectStats_2016Q2.csv.gz",skiprows=1)

#reject_2016Q3 = pd.read_csv("data/rejected/2016Q3/RejectStats_2016Q3.csv.gz",skiprows=1)

#reject_2016Q4 = pd.read_csv("data/rejected/2016Q4/RejectStats_2016Q4.csv.gz",skiprows=1)




In [ ]:
reject_2007_2012.columns

In [ ]:
reject_2007_2012.describe()